In [1]:
import numpy as np

def heston_charfunc(phi, S0, T, r, kappa, theta, sigma, rho, v0, j=1):
    """
    Calcule la fonction caractéristique pour j=1 ou j=2 
    selon la formule Heston, c.-à-d. alpha = j-1.
    On suppose dividende=0 pour simplifier (sinon on remplace r par (r-q)).

    phi : float (intégration sur [0,∞), phi >= 0)
    S0 : spot
    T  : maturité
    r  : taux sans risque
    kappa, theta, sigma, rho, v0 : paramètres Heston
    j  : 1 ou 2
    """

    alpha = j - 1.0  # alpha=0 pour j=1, alpha=1 pour j=2

    # a, b param. standard
    a = kappa * theta
    # selon la littérature, on a b = kappa - rho*sigma si j=1, 
    # ou b = kappa si j=2. 
    # Mais il existe plusieurs conventions. Voici l'une des plus courantes :
    if j == 1:
        u = 0.5
        b = kappa - rho * sigma
    else:  # j=2
        u = -0.5
        b = kappa

    # On définit d = sqrt( (rho*sigma*i*phi - b)^2 + sigma^2 * (i*phi + phi^2) )
    i_phi = phi * 1j
    d = np.sqrt( (rho*sigma*i_phi - b)**2 + (sigma**2)*(u + i_phi)**2 )

    # g = (b - rho*sigma*i*phi + d) / (b - rho*sigma*i*phi - d)
    g = (b - rho*sigma*i_phi + d) / (b - rho*sigma*i_phi - d)

    # Term1: exp(i*phi * ln(S0) ) * exp(i*phi * r*T ) 
    # (ici, pas de dividende => juste r)
    C = np.log(S0) * i_phi + (r * i_phi)*T

    # Term2: (dT) = d * T
    # Pour la partie exponentielle "D(t)" au sens Heston
    # On a un factor: (1 - g*exp(dT)) / (1 - g)
    # et exp(...) * v0
    # etc.
    # Écriture standard :
    numerator   = 1.0 - np.exp(d * T) * g
    denominator = 1.0 - g
    # force un tout petit epsilon si denominator ~ 0 => num instable
    # On peut faire:
    numerator   = np.where(np.abs(numerator) < 1e-14, 1e-14, numerator)
    denominator = np.where(np.abs(denominator) < 1e-14, 1e-14, denominator)

    # Log of that ratio
    log_term = np.log(numerator/denominator)

    # big_A
    # A = i*phi * ...
    # Sur la base du papier de Heston ou d'autres refs, on a:
    # A(t) = (kappa * theta / sigma^2) * ( (b - rho*sigma i phi + d)*T - 2*log( (1-g e^{dT}) / (1-g) ) )
    # (en ajustant pour "u" etc.)
    # Les formules varient en fonction de la convention. 
    # En voici une version "classique" qu'on voit souvent :
    A = (i_phi*u - phi**2*0.5) * T
    A += (a / sigma**2) * ( (d - (b - rho*sigma*i_phi)) * T - 2.0 * log_term )

    # big_B
    # B = ...
    # B(t) = (b - rho*sigma i phi + d)/ sigma^2 * (1 - exp(dT)) / (1 - g e^{dT})
    B = (d - (b - rho*sigma*i_phi)) / sigma**2
    B *= (1.0 - np.exp(d * T)) / (1.0 - g * np.exp(d * T))

    # final charfunc
    # log char => C + A + v0 * B
    # => varphi_j(phi) = exp( C + A + v0*B )
    return np.exp(C + A + v0*B)


In [2]:
import numpy as np
from scipy.integrate import quad

def P_j(j, S0, K, T, r, kappa, theta, sigma, rho, v0):
    """Calcule le P_j via l'intégrale Heston."""
    # On définit l'intégrande:
    def integrand(phi):
        # phi est reel >= 0
        i_phi = 1j*phi
        # charfunc:
        varphi = heston_charfunc(phi, S0, T, r, kappa, theta, sigma, rho, v0, j=j)
        # e^{-i phi ln(K)} 
        e_term = np.exp(-i_phi * np.log(K))
        # fraction:
        fraction = e_term * varphi / (i_phi)
        return np.real(fraction)
    
    # On intègre de 0 à +∞. En pratique, on tronque souvent à [0, xMax].
    xMax = 100.0  # tu peux essayer 50, 200, etc.
    result, _ = quad(integrand, 0, xMax, limit=200)
    return 0.5 + (1.0/np.pi)*result


In [3]:
def heston_price_call(S0, K, T, r, kappa, theta, sigma, rho, v0):
    p1 = P_j(j=1, S0=S0, K=K, T=T, r=r, kappa=kappa, theta=theta, sigma=sigma, rho=rho, v0=v0)
    p2 = P_j(j=2, S0=S0, K=K, T=T, r=r, kappa=kappa, theta=theta, sigma=sigma, rho=rho, v0=v0)
    call_price = S0 * p1 - K * np.exp(-r*T) * p2
    return call_price


In [4]:
def objective_heston_semi_closed(params, df_calls, S0, r):
    kappa, theta, rho, sigma, v0 = params  # (on suppose 5 params)
    # Vérifie la validité
    if kappa <= 0 or theta <= 0 or sigma <= 0 or v0 <= 0 or not -1<rho<1:
        return 1e10
    
    errors = []
    for idx, row in df_calls.iterrows():
        K_i = row["strike"]
        P_market = row["lastPrice"]
        T_i = row["T"]

        P_model = heston_price_call(S0, K_i, T_i, r, kappa, theta, sigma, rho, v0)
        errors.append( (P_model - P_market)**2 )
    
    return np.mean(errors)


In [5]:
from scipy.optimize import minimize

def calibrate_heston_semi_closed(df_calls, S0, r):
    init_guess = [1.0, 0.02, -0.5, 0.3, 0.02]
    bounds = [(1e-6, 10),
              (1e-6, 1),
              (-0.999, 0.999),
              (1e-6, 3),
              (1e-6, 1)]
    
    res = minimize(objective_heston_semi_closed,
                   x0=init_guess,
                   args=(df_calls, S0, r),
                   method='L-BFGS-B',
                   bounds=bounds)
    return res.x, res.fun


In [8]:
# Suppose qu'on ait les fonctions déjà définies :
#  - heston_charfunc(...)
#  - P_j(...)
#  - heston_price_call(...)
#  - objective_heston_semi_closed(...)
#  - calibrate_heston_semi_closed(...)

# 1) On choisit des paramètres "exemple"
S0_test   = 100.0
K_test    = 100.0
T_test    = 1.0     # 1 an
r_test    = 0.03    # 3%
kappa_test= 1.0
theta_test= 0.02
sigma_test= 0.2
rho_test  = -0.5
v0_test   = 0.02

# 2) On appelle la fonction de pricing
price_test = heston_price_call(
    S0_test, 
    K_test,
    T_test,
    r_test,
    kappa_test,
    theta_test,
    sigma_test,
    rho_test,
    v0_test
)

print("Prix du call (Heston) avec paramètres test :", price_test)


Prix du call (Heston) avec paramètres test : 14.660484033770537


In [12]:
import yfinance as yf
import pandas as pd

ticker = yf.Ticker("AAPL")
# Disons qu'on prend une date d'expiration un peu lointaine (mais la plus proche est souvent plus liquide)
chosen_expiration = ticker.options[10]  # Par ex. la 1ère ou "2024-12-20"
chain = ticker.option_chain(chosen_expiration)

df_calls_raw = chain.calls.copy()
df_calls_raw["expiration"] = chosen_expiration

# On calcule la maturité T en années
import datetime
current_date = pd.to_datetime("today")
expiration_date = pd.to_datetime(chosen_expiration)

df_calls_raw["T"] = (expiration_date - current_date).days / 365.0

In [23]:
hist = ticker.history(period="1d")
S0 = hist["Close"].iloc[-1]
r = 0.04  # Taux sans risque "bidon" pour l'exemple

df_calls_raw_sample = df_calls_raw.sample(10)

In [19]:
print(S0)

222.63999938964844


In [24]:
best_params, min_error = calibrate_heston_semi_closed(df_calls_raw_sample, S0, r)
print("Paramètres optimaux :", best_params)
print("Erreur moyenne :", min_error)

C:\Users\gabri\AppData\Local\Temp\ipykernel_23444\664930710.py:20: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  result, _ = quad(integrand, 0, xMax, limit=200)
C:\Users\gabri\AppData\Local\Temp\ipykernel_23444\664930710.py:20: IntegrationWarning: The integral is probably divergent, or slowly convergent.
  result, _ = quad(integrand, 0, xMax, limit=200)
C:\Users\gabri\AppData\Local\Temp\ipykernel_23444\664930710.py:20: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  result, _ = quad(integrand, 0, xMax, limit=200)
C:\Users\gabri\AppData\Local\Temp\ipykernel_23444\664930710.py:20: IntegrationWarning: The integral is probably divergent, or slowly convergent.
  result, _ = quad(integrand, 0, xMax, limit=200)
C:\Users\gabri\AppData\Local\Temp\ipykernel_

Paramètres optimaux : [ 1.20704645  0.70683777 -0.76775838  1.21455007  0.2074539 ]
Erreur moyenne : 82.14000389532421


In [25]:
kappa_star, theta_star, rho_star, sigma_star, v0_star = best_params


In [26]:
price_call = heston_price_call(
    S0,           # spot actuel d'AAPL
    210,            # le strike qui t'intéresse
    0.5,            # la maturité souhaitée (en années)
    0.04,            # taux sans risque
    kappa_star,
    theta_star,
    sigma_star,
    rho_star,
    v0_star
)


In [27]:
price_call

np.float64(34.31166358905068)

In [22]:
df_calls_raw

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency,expiration,T
0,AAPL250718C00120000,2025-01-03 20:25:30+00:00,120.0,125.90,0.00,0.00,0.0,0.0,1.0,0,0.000010,True,REGULAR,USD,2025-07-18,0.482192
1,AAPL250718C00125000,2025-01-15 15:54:46+00:00,125.0,116.00,0.00,0.00,0.0,0.0,3.0,0,0.000010,True,REGULAR,USD,2025-07-18,0.482192
2,AAPL250718C00130000,2025-01-08 20:36:11+00:00,130.0,115.35,0.00,0.00,0.0,0.0,3.0,0,0.000010,True,REGULAR,USD,2025-07-18,0.482192
3,AAPL250718C00135000,2025-01-15 18:02:09+00:00,135.0,105.53,0.00,0.00,0.0,0.0,1.0,0,0.000010,True,REGULAR,USD,2025-07-18,0.482192
4,AAPL250718C00145000,2025-01-21 16:20:31+00:00,145.0,79.22,0.00,0.00,0.0,0.0,2.0,0,0.000010,True,REGULAR,USD,2025-07-18,0.482192
5,AAPL250718C00150000,2025-01-21 16:49:34+00:00,150.0,74.10,0.00,0.00,0.0,0.0,1.0,0,0.000010,True,REGULAR,USD,2025-07-18,0.482192
6,AAPL250718C00155000,2024-11-26 19:04:41+00:00,155.0,85.50,104.60,107.10,0.0,0.0,NaN,2,1.326419,True,REGULAR,USD,2025-07-18,0.482192
7,AAPL250718C00160000,2025-01-21 17:09:28+00:00,160.0,65.35,0.00,0.00,0.0,0.0,2.0,0,0.000010,True,REGULAR,USD,2025-07-18,0.482192
8,AAPL250718C00165000,2024-12-17 14:31:28+00:00,165.0,90.50,68.85,69.35,0.0,0.0,1.0,9,0.610050,True,REGULAR,USD,2025-07-18,0.482192
9,AAPL250718C00170000,2025-01-16 19:27:01+00:00,170.0,64.42,0.00,0.00,0.0,0.0,5.0,0,0.000010,True,REGULAR,USD,2025-07-18,0.482192
